In [1]:
from pl_bolts.models.self_supervised.simclr.simclr_module import SimCLR
from data_loaders.data_module import ChestDataModule
import pytorch_lightning as pl
from transforms.pretraining import SimCLREvalDataTransform, SimCLRTrainDataTransform
from pytorch_lightning.loggers import WandbLogger
import torch

In [2]:
dm = ChestDataModule(['chest14'], return_dict=False)
dm.train_transforms = SimCLRTrainDataTransform(input_height=128)
dm.val_transforms = SimCLREvalDataTransform(input_height=128)

Loaded datasets: chest14


In [3]:
train_size = dm.get_size("train")

In [4]:
train_size

69219

In [5]:
model = SimCLR(gpus=1, batch_size=16, num_samples=train_size, dataset='chest14', learning_rate=0.0001, warmup_epochs=1)

In [6]:
print(model)

SimCLR(
  (encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (

In [7]:
wandb_logger = WandbLogger(name='simclr_chest14',project='thesis')
# checkpoint_callback = ModelCheckpoint(monitor='val_loss', 
#                                       dirpath='logs/baseline/pneumo/', 
#                                       filename='resnet-NL-0.1-Adam-3e_6-{epoch:02d}-{val_loss:.4f}')

trainer = pl.Trainer(gpus=1, deterministic=True,
                     logger=wandb_logger, callbacks=[])#checkpoint_callback])

if torch.cuda.is_available():
    model = model.cuda()

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [8]:
trainer.fit(model, dm)

wandb: Currently logged in as: genvekt (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.19 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade



  | Name       | Type       | Params
------------------------------------------
0 | encoder    | ResNet     | 25.6 M
1 | projection | Projection | 4.5 M 
------------------------------------------
30.0 M    Trainable params
0         Non-trainable params
30.0 M    Total params


Before sampling length:  17305
After sampling length:  17305


/home/intern/.local/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/intern/.local/lib/python3.7/site-packages/pytorch_lightning/core/step_result.py:145: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = torch.tensor(value, device=device, dtype=torch.float)


Before sampling length:  69219
After sampling length:  69219


/home/intern/.local/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/intern/.local/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [1]:
import pandas as pd

In [10]:
df = pd.read_csv("datasets/chexpert.csv")

In [11]:
df[df['AP/PA']=="AP"]

,Unnamed: 0,Unnamed: 0.1,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,...,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,patient,Target,Phase
1,80401,80784,/new_data/CheXpert/CheXpert-v1.0/train/patient...,Male,51,Frontal,AP,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,1.0,NaN,patient19397,1,train
2,80402,80785,/new_data/CheXpert/CheXpert-v1.0/train/patient...,Male,51,Frontal,AP,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,patient19397,1,train
3,78058,78436,/new_data/CheXpert/CheXpert-v1.0/train/patient...,Female,86,Frontal,AP,NaN,0.0,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,patient18829,1,train
4,78059,78437,/new_data/CheXpert/CheXpert-v1.0/train/patient...,Female,86,Frontal,AP,NaN,0.0,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,patient18829,1,train
8,92712,93186,/new_data/CheXpert/CheXpert-v1.0/train/patient...,Male,66,Frontal,AP,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,-1.0,NaN,patient22349,1,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190045,75280,75649,/new_data/CheXpert/CheXpert-v1.0/train/patient...,Male,24,Frontal,AP,NaN,NaN,NaN,...,NaN,NaN,1.0,1.0,NaN,NaN,1.0,patient18152,1,val
190046,75281,75650,/new_data/CheXpert/CheXpert-v1.0/train/patient...,Male,24,Frontal,AP,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,patient18152,1,val
190047,211321,212416,/new_data/CheXpert/CheXpert-v1.0/train/patient...,Male,42,Frontal,AP,1.0,0.0,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,patient56590,0,val
190048,7570,7602,/new_data/CheXpert/CheXpert-v1.0/train/patient...,Male,62,Frontal,AP,NaN,NaN,0.0,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,patient01869,1,val


In [12]:
df.shape

(190051, 24)

In [13]:
csv_data = df
csv_data = csv_data[(csv_data["Atelectasis"].isin([1.0,-1.0])) | (csv_data["Cardiomegaly"].isin([1.0,-1.0])) | (csv_data["Consolidation"].isin([1.0,-1.0])) | (csv_data["Edema"].isin([1.0,-1.0])) | (csv_data["Pleural Effusion"].isin([1.0,-1.0]))]

In [15]:
csv_data[csv_data['AP/PA'] == 'AP']

,Unnamed: 0,Unnamed: 0.1,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,...,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,patient,Target,Phase
1,80401,80784,/new_data/CheXpert/CheXpert-v1.0/train/patient...,Male,51,Frontal,AP,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,1.0,NaN,patient19397,1,train
8,92712,93186,/new_data/CheXpert/CheXpert-v1.0/train/patient...,Male,66,Frontal,AP,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,-1.0,NaN,patient22349,1,train
11,69733,70074,/new_data/CheXpert/CheXpert-v1.0/train/patient...,Male,78,Frontal,AP,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,patient16797,1,train
12,69734,70075,/new_data/CheXpert/CheXpert-v1.0/train/patient...,Male,83,Frontal,AP,NaN,NaN,1.0,...,NaN,-1.0,NaN,NaN,NaN,NaN,1.0,patient16797,1,train
13,69735,70076,/new_data/CheXpert/CheXpert-v1.0/train/patient...,Male,83,Frontal,AP,NaN,NaN,NaN,...,NaN,NaN,0.0,1.0,NaN,NaN,1.0,patient16797,1,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190040,8709,8741,/new_data/CheXpert/CheXpert-v1.0/train/patient...,Male,56,Frontal,AP,NaN,1.0,NaN,...,NaN,-1.0,NaN,NaN,NaN,NaN,1.0,patient02154,1,val
190041,8710,8742,/new_data/CheXpert/CheXpert-v1.0/train/patient...,Male,56,Frontal,AP,NaN,NaN,0.0,...,NaN,1.0,0.0,1.0,NaN,NaN,1.0,patient02154,1,val
190045,75280,75649,/new_data/CheXpert/CheXpert-v1.0/train/patient...,Male,24,Frontal,AP,NaN,NaN,NaN,...,NaN,NaN,1.0,1.0,NaN,NaN,1.0,patient18152,1,val
190048,7570,7602,/new_data/CheXpert/CheXpert-v1.0/train/patient...,Male,62,Frontal,AP,NaN,NaN,0.0,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,patient01869,1,val
